# Visualizing Flood Data as Cluster Markers

When plotting lots of polygons, it is very slow. Make it faster by plotting as points/markers.

`FastMarkerCluster` requires Folium 0.10.1, which is not in pypi. Install manually through this tutorial

https://github.com/python-visualization/folium/issues/1089

In [ ]:
# !pip install git+https://github.com/python-visualization/folium.git

In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd

import seaborn as sns
import folium

from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
file_folder = 'DOAE spatial data/ภัยน้ำท่วม/'

file_list = glob.glob(file_folder + '*.shp')
file_list

In [ ]:
file_list.sort()
file_list

In [ ]:
data = gpd.read_file(file_list[0], encoding='utf-8')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

## Show as Cluster Markers

In [ ]:
data.iloc[0].geometry

In [ ]:
type(data.iloc[0].geometry.centroid.coords.xy)

In [ ]:
x,y = data.iloc[0].geometry.centroid.coords.xy
print(x[0], y[0])

In [ ]:
def get_centroid(row):
    x,y = row.geometry.centroid.coords.xy
    return [y[0], x[0]]

In [ ]:
MAX_RECORDS = 1000

In [ ]:
locations = data[:MAX_RECORDS].apply(get_centroid, axis=1)
locations[:10].values.tolist()

In [ ]:
m = folium.Map(location=[14.069446, 100.603271], zoom_start=10)
cluster = FastMarkerCluster(locations[:MAX_RECORDS].values.tolist())
m.add_child(cluster)
m

Use callback with javascript syntax to modify markers

https://leafletjs.com/

In [ ]:
def get_centroid(row):
    x,y = row.geometry.centroid.coords.xy
    return [y[0], x[0], row.gid]

In [ ]:
callback = """\
function (row) {
    var marker;
    marker = L.marker(new L.LatLng(row[0], row[1])).bindPopup('gid: ' + row[2].toString());
    return marker;
};
"""

m = folium.Map(location=[14.069446, 100.603271], zoom_start=10)
cluster = FastMarkerCluster(locations[:MAX_RECORDS].values.tolist(), callback=callback)
m.add_child(cluster)
m

## Apply to all files

In [ ]:
m = folium.Map(location=[14.069446, 100.603271], zoom_start=10)

MAX_RECORDS = 1000
for f in sorted(file_list)[:2]:
    print('Read: ' + f)
    data = gpd.read_file(f, encoding='utf-8')
    locations = data[:MAX_RECORDS].apply(get_centroid, axis=1)
    
    cluster = FastMarkerCluster(locations[:MAX_RECORDS].values.tolist(), callback=callback, name=os.path.basename(f))
    m.add_child(cluster)

folium.LayerControl().add_to(m)
m

In [ ]:
m.save('./output/flood_62.html')